In [1]:
cd /home/nampv1/projects/vnpost_asr/

/home/nampv1/projects/vnpost_asr


In [2]:
models_list =[
    "/home/nampv1/models/asr/vnpost-1",
    "/home/nampv1/models/asr/vnpost-2",
    "/home/nampv1/models/asr/vnpost-3",
]

In [3]:
from src.utils.model_utils import load_model

/home/nampv1/anaconda3/envs/asr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
config_path = "configs/transcribe.yaml"

In [5]:
from omegaconf import OmegaConf

In [6]:
from src.utils.exp_utils import setup_environment

In [7]:
from src.utils.model_utils import load_model, load_processor

In [8]:
setup_environment()

In [9]:
cfg = OmegaConf.load(config_path)

In [10]:
print(OmegaConf.to_yaml(cfg))

model:
  model_type: asr
  pretrained_model_name_or_path: openai/whisper-small
  pretrained_processor_name_or_path: null
  load_in_4bit: true
  load_in_8bit: true
  bnb_4bit_compute_dtype: null
  bnb_4bit_quant_type: nf4
  bnb_4bit_use_double_quant: false
  bnb_4bit_quant_storage: uint8
  torch_dtype: bfloat16
  attn_implementation: null
  device_map: null
  low_cpu_mem_usage: null
  adapter_path: null
input:
  audio_path: data/examples/audio.wav
generate: null
device:
  use_cpu: false



In [11]:
model_args = cfg.model 
device_args = cfg.device

In [12]:
device_args

{'use_cpu': False}

In [25]:
model_args.pretrained_model_name_or_path = "/home/nampv1/models/asr/vnpost-2/snapshots/e37978b90ca9030d5170a5c07aadb050351a65bb"

In [26]:
print(OmegaConf.to_yaml(model_args))

model_type: asr
pretrained_model_name_or_path: /home/nampv1/models/asr/vnpost-2/snapshots/e37978b90ca9030d5170a5c07aadb050351a65bb
pretrained_processor_name_or_path: null
load_in_4bit: true
load_in_8bit: true
bnb_4bit_compute_dtype: null
bnb_4bit_quant_type: nf4
bnb_4bit_use_double_quant: false
bnb_4bit_quant_storage: uint8
torch_dtype: bfloat16
attn_implementation: null
device_map: null
low_cpu_mem_usage: null
adapter_path: null



In [27]:
model = load_model(model_args=model_args, device_args=device_args)

In [28]:
processor = load_processor(model_args)

In [29]:
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear4bit(in_features=512, out_features=512, bias=False)
            (v_proj): Linear4bit(in_features=512, out_features=512, bias=True)
            (q_proj): Linear4bit(in_features=512, out_features=512, bias=True)
            (out_proj): Linear4bit(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear4bit(in_features=512, out_features=2048, bias=True)
          (fc2): Linear4bit(in_features=2048, out_features=512, b

In [30]:
adapter_path = "/home/nampv1/models/asr/vnpost-1/"
base_model = model

from peft import PeftModel
finetuned_model = PeftModel.from_pretrained(base_model, adapter_path)

finetuned_model = finetuned_model.merge_and_unload()

/home/nampv1/anaconda3/envs/asr/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [31]:
finetuned_model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear4bit(in_features=512, out_features=512, bias=False)
            (v_proj): Linear4bit(in_features=512, out_features=512, bias=True)
            (q_proj): Linear4bit(in_features=512, out_features=512, bias=True)
            (out_proj): Linear4bit(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear4bit(in_features=512, out_features=2048, bias=True)
          (fc2): Linear4bit(in_features=2048, out_features=512, b